In [5]:
print("hellow world")

import requests

hellow world


In [6]:
url = "http://host.docker.internal:11434/api/generate"
payload = {
    "model": "llama3:8b",       # замените на нужную модель
    "prompt": "Напиши очень при очень краткое резюме по теме 'формальная верификация в coq' на русском языке",
    "stream": False
}

In [7]:
resp = requests.post(url, json=payload, timeout=120)

In [8]:
resp.raise_for_status()
data = resp.json()  # вернёт JSON с результатом
print(data)

{'model': 'llama3:8b', 'created_at': '2025-08-25T06:38:21.089615781Z', 'response': 'Резюме:\n\nФормальная верификация - процесс проверки программы на отсутствие логических ошибок, используя формальные методы и инструменты. В Coq это достигается с помощью так называемых "proofs" (доказательств), которые позволяют доказать истинность утверждения о поведении программы.\n\nВ Coq proof assistant есть несколько ключевых функциональных возможностей:\n\n* Система типов - позволяет определить и проверить тип данных для каждого выражения в программе.\n* Функции отображения - позволяют определить отношения между выражениями, что важно при доказательстве свойств программы.\n* Направленное доказательство - процесс доказательства, направленный на то, чтобы доказать или опровергнуть некоторое утверждение о поведении программы.\n\nС помощью этих функциональных возможностей в Coq можно формально проверить соответствие программы ее спецификации, а также доказать истиныность свойств программы, таких как 

In [104]:
import requests
import json
from typing import Optional

API_BASE = "http://host.docker.internal:11434"  # Базовый URL API Ollama
GENERATE_PATH = "/api/generate"  # Путь для генерации текста

# Кастомное исключение для ошибок работы с Ollama API
class OllamaAPIError(Exception):
    pass

def check_server() -> None:
    """
    Проверяет доступность сервера Ollama.
    Бросает исключение, если сервер недоступен или ответ не соответствует ожиданиям.
    """
    try:
        r = requests.get(API_BASE + "/", timeout=5)  # Запрос к корневому пути
    except requests.RequestException as e:
        # Если соединение не удалось
        raise OllamaAPIError(f"Cannot reach {API_BASE}: {e}")
    # Проверка статуса и содержимого ответа
    if r.status_code != 200 or "Ollama" not in (r.text or ""):
        raise OllamaAPIError(f"Service at {API_BASE} responded unexpectedly: status={r.status_code}, body={r.text!r}")

def extract_text_from_json(obj) -> Optional[str]:
    """
    Извлекает текст из JSON-объекта, который может содержать ответ модели.
    Поддерживает разные форматы (OpenAI-like, Ollama-like).
    """
    if obj is None:
        return None
    if isinstance(obj, str):
        return obj
    if isinstance(obj, dict):
        # Проверка распространённых ключей
        for key in ("response", "text", "completion", "content"):
            if key in obj and isinstance(obj[key], str):
                return obj[key]
        # Обработка формата с choices[]
        ch = obj.get("choices")
        if isinstance(ch, list) and ch:
            first = ch[0]
            if isinstance(first, dict):
                # OpenAI streaming формат: delta.content
                if "delta" in first and isinstance(first["delta"], dict):
                    return first["delta"].get("content")
                # Прямой текст
                if "text" in first and isinstance(first["text"], str):
                    return first["text"]
                # Формат message.content
                if "message" in first and isinstance(first["message"], dict):
                    return first["message"].get("content")
    return None

def stream_until_dot(goal_text: str, failed_tactics = [], model: str = "llama3:8b", timeout: int = 120) -> str:
    """
    Отправляет запрос к Ollama API с целью сгенерировать одну строку тактики Coq.
    Читает потоковый ответ до первой точки '.' и возвращает собранный текст.
    
    :param goal_text: Цель в синтаксисе Coq, для которой требуется тактика
    :param model: Название модели (по умолчанию llama3:8b)
    :param timeout: Таймаут запроса в секундах
    :return: Сгенерированная тактика (строка, заканчивающаяся точкой)
    """
    check_server()  # Проверка доступности сервера

    url = API_BASE + GENERATE_PATH
    # Формируем инструкцию для модели
    prompt = f"""You are a Coq tactic suggester.

INPUT:
A Coq goal is provided below in its ORIGINAL Coq format. Do not modify or repeat it.
A list of previously tried tactics that FAILED is also provided; avoid using them again.

GOAL:
```coq
{goal_text}
````

FAILED TACTICS:
{failed_tactics}

TASK:
Output EXACTLY ONE Coq tactic line that makes the most direct progress toward solving the goal and is NOT in the failed tactics list.

HARD RULES:

* Output ONLY a single Coq tactic line terminated by a period.
* No explanations, no prefixes, no code fences, no extra text, no alternatives.
* Do NOT repeat or alter the goal.
* Prefer the most direct, type-correct next step.
* If multiple options exist, choose the single most straightforward one.
* Semicolons (`;`) and proof brackets (`[...]`) are allowed, but the result must be ONE line.
* Never output a tactic that appears in the FAILED TACTICS list.
* Never answer "I don't know" or similar; always output your best single tactic.
* Keep it minimal.
* At the end it is necessary to have `.`
* allowed `auto.`, `reflexivity.`, `trivial.`, `subst.`

Respond with ONLY the tactic line."""
    # Тело запроса для Ollama API
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True  # Включаем потоковый режим
    }
    
    # Отправка запроса
    resp = requests.post(url, json=payload, stream=True, timeout=timeout)
    if resp.status_code == 404:
        # Ошибка 404 — сервер не настроен или не запущен
        resp.close()
        raise OllamaAPIError(
            "404 from /api/generate. Проверьте: контейнер запущен ли с 'ollama serve', доступен ли порт 11434, и не занят ли порт другим процессом. "
            "Выполните `curl http://localhost:11434/` и `docker logs ollama`."
        )
    resp.raise_for_status()  # Проверка на другие HTTP-ошибки
    
    collected = ""  # Буфер для символов
    try:
        # Читаем поток построчно
        for raw_line in resp.iter_lines(decode_unicode=True):
            if raw_line is None:
                continue
            line = raw_line.strip()
            if not line:
                continue
            # Попытка распарсить строку как JSON
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                # Если не JSON — используем как текст
                text = line
            else:
                # Извлекаем полезный текст из объекта
                text = extract_text_from_json(obj) or ""
            if not text:
                continue
            # Ищем первую точку в ответе
            for ch in text:
                if ch == ".":
                    # Закрываем соединение и возвращаем собранный ответ
                    resp.close()
                    return collected.strip() + "."
                collected += ch
        # Если поток закончился без точки — возвращаем что есть
        resp.close()
        return collected.strip()
    finally:
        # Гарантированное закрытие соединения
        try:
            resp.close()
        except Exception:
            pass


In [87]:
for i in range(20):
        print (stream_until_dot(
    """1 goal
l : list nat
Hall : forall x : nat, In x l
______________________________________(1/1)
In (S (max_list l)) l
    """), "\n______________________________________\n\n\n")

decide. 
______________________________________



rewrite <- Hall. 
______________________________________



unfold max_list, fold_left nat_add; 
______________________________________



decide. 
______________________________________



decision. 
______________________________________



constructor; 
______________________________________



simplify. 
______________________________________



unfold_in_map. 
______________________________________



auto. 
______________________________________



intro H. 
______________________________________



constructor. 
______________________________________



congruence. 
______________________________________



constr_solve. 
______________________________________



destruct l as [x L]. 
______________________________________



destruct l as [x xs]. 
______________________________________



constructive_or. 
______________________________________



independent. 
______________________________________



instantiate (1). 
_______

In [7]:
!pip install sexpdata

In [109]:
import subprocess, sexpdata
from sexpdata import loads
import select

def read(proc):
    timeout = 5  # секунд
    ready, _, _ = select.select([proc.stdout], [], [], timeout)
    if not ready:
        return "eror"
    
    line = "("
    while "Completed" not in line:
        line += proc.stdout.readline()
        #print(line.split("\n")[-2])
    return line + ")"

def Add(proc, line):
    cmd = f'(Add () "{line}")\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def Exec(proc, i):
    cmd = f'(Exec {i})\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def Cancel(proc, i):
    cmd = f'(Cancel ({i}))\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def Query(proc, i):
    cmd = f'(Query ((sid {i}) (pp ((pp_format PpStr)))) Goals)\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

def find_a_leters(tree, leters, path="0"):
    results = []
    if isinstance(tree, list) or isinstance(tree, tuple):
        if tree and isinstance(tree[0], sexpdata.Symbol) and tree[0].value() in leters:
            cur_tree = []
            for i, elem in enumerate(tree):
                if i != 0 and elem and isinstance(elem, list) \
                    and isinstance(elem[0], sexpdata.Symbol) and elem[0].value() in leters:
                    break
                cur_tree.append(elem)
            results.append((path, cur_tree))
        for i, elem in enumerate(tree):
            results += find_a_leters(elem, leters, path + "." + str(i))
    return results

def solve(lemma):
    proc = subprocess.Popen(
        ["sertop"],
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    line = proc.stdout.readline()
    while "(contents Processed)" not in line:
        line = proc.stdout.readline()

    added = 2
    Add(proc, str(lemma))
    Exec(proc, added)
    ans = find_a_leters(loads(Query(proc, added)), "CoqString")[0][1][1]
    print(ans)
    print("============================")

    failed_tactics = []

    while ans:
        tactic = stream_until_dot(ans, failed_tactics)
        #tactic = "some eror.."
        if "." != tactic[-1] : tactic += '.'
        print(tactic)
        Add(proc, tactic)
        ex = find_a_leters(loads(Exec(proc, added + 1)), "CoqExn")
        if not ex:
            added += 1
            failed_tactics = []
            
            ans = find_a_leters(loads(Query(proc, added)), "CoqString")[0][1][1]
            print(ans)
            print("============================")
        else:
            print("eror")
            print("============================")
            Cancel(proc, added + 1)
            failed_tactics.append(tactic)

    print("Qed.")
    

In [113]:
solve("Lemma test: forall (x y z : nat), x = y -> y = z -> x = z.")

none
forall x y z : nat,
                             x = y -> y = z -> x = z
rewrite -> 1.
eror
intros.

  H0 : y = z
  H : x = y
  x, y, z : nat
x = z
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H0 in |-*.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H0 in H.
eror
rewrite H in H0.
eror
rewrite H0 in |-*.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror
rewrite H in H0.
eror


KeyboardInterrupt: 

In [44]:
proc = subprocess.Popen(
    ["sertop"],
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)
line = proc.stdout.readline()
while "(contents Processed)" not in line:
    line = proc.stdout.readline()

Add(proc, "Lemma test: forall (x y : nat), x = y -> y = x.")

'((Answer 0 Ack)\n(Answer 0(Added 2((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 36))NewAddTip))\n(Answer 0 Completed)\n)'

In [46]:
Add(proc, "Proof.")

'((Answer 2 Ack)\n(Answer 2(Added 4((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 6))NewAddTip))\n(Answer 2 Completed)\n)'

In [13]:
Add(proc, "intros x y H; rewrite H.")

'((Answer 40 Ack)\n(Answer 40(Added 5((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 9))NewAddTip))\n(Answer 40 Completed)\n)'

In [5]:
Add(proc, "auto.")

'((Answer 2 Ack)\n(Answer 2(Added 5((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 5))NewAddTip))\n(Answer 2 Completed)\n)'

In [10]:
print(Cancel(proc, 4))

((Answer 27 Ack)
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Answer 27(Canceled(4)))
(Answer 27 Completed)
)


In [47]:
for i in range(1, 7):
    print(Exec(proc, i), "\n")

((Answer 3 Ack)
(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))
(Answer 3 Completed)
) 

((Answer 4 Ack)
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Answer 4 Completed)
) 

((Answer 5 Ack)
(Feedback((doc_id 0)(span_id 3)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Answer 5 Completed)
) 

((Answer 6 Ack)
(Feedback((doc_id 0)(span_id 4)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))
(Answer 6 Completed)
) 

((Answer 7 Ack)
(Answer 7(CoqExn((loc())(stm_ids())(backtrace(Backtrace()))(exn(Stm.Vcs_aux.Expired))(pp(Pp_glue((Pp_box(Pp_hovbox 0)(Pp_glue((Pp_string Anomaly)(Pp_print_br

In [15]:
for i in range(1, 7):
    print(Query(proc, i), "\n")

((Answer 47 Ack)
(Answer 47(ObjList()))
(Answer 47 Completed)
) 

((Answer 48 Ack)
(Answer 48(ObjList((CoqString"none\n============================\nforall x : nat, x = x"))))
(Answer 48 Completed)
) 

((Answer 49 Ack)
(Answer 49(ObjList((CoqString"none\n============================\nforall x : nat, x = x"))))
(Answer 49 Completed)
) 

((Answer 50 Ack)
(Answer 50(ObjList()))
(Answer 50 Completed)
) 

((Answer 51 Ack)
(Answer 51(ObjList((CoqString"\n  x : nat\n============================\nx = x"))))
(Answer 51 Completed)
) 

((Answer 52 Ack)
(Answer 52(ObjList()))
(Answer 52 Completed)
) 



In [69]:
i = 1
while "eror" not in Exec(i) and "CoqExn" not in Exec(i):
    i += 1
    print(i, end="")

234